# Lab 3 Exercise 3
## Binary Classification

We are given a dataset that consists of biological features of a person and whether or not they are a smoker.

There are *22 descriptive features* and *1 target column* in the dataset as described in the Table below. You are tasked with creating a predictive model to predict the column "SMK_stat_type_cd", which specifies smoker/non-smoker status of the person. You are expected to use classification methods that was shown in the previous exercises.



|      Column      |                                 Description                                 |
|:----------------:|:---------------------------------------------------------------------------:|
| identified_gender              | male, female                                                                |
| age              | round up to 5 years                                                         |
| height           | round up to 5 cm[cm]                                                        |
| weight           | [kg]                                                                        |
| sight_left       | eyesight(left)                                                              |
| sight_right      | eyesight(right)                                                             |
| hear_left        | hearing left, 1(normal), 2(abnormal)                                        |
| hear_right       | hearing right, 1(normal), 2(abnormal)                                       |
| SBP              | Systolic blood pressure[mmHg]                                               |
| DBP              | Diastolic blood pressure[mmHg]                                              |
| BLDS             | BLDS or FSG(fasting blood glucose)[mg/dL]                                   |
| tot_chole        | total cholesterol[mg/dL]                                                    |
| HDL_chole        | HDL cholesterol[mg/dL]                                                      |
| LDL_chole        | LDL cholesterol[mg/dL]                                                      |
| triglyceride     | triglyceride[mg/dL]                                                         |
| hemoglobin       | hemoglobin[g/dL]                                                            |
| urine_protein    | protein in urine, 1(-), 2(+/-), 3(+1), 4(+2), 5(+3), 6(+4)                  |
| serum_creatinine | serum(blood) creatinine[mg/dL]                                              |
| SGOT_AST         | SGOT(Glutamate-oxaloacetate transaminase) AST(Aspartate transaminase)[IU/L] |
| SGOT_ALT         | ALT(Alanine transaminase)[IU/L]                                             |
| gamma_GTP        | y-glutamyl transpeptidase[IU/L]                                             |
| SMK_stat_type_cd **(Target)** | Smoking state, 0(never), 1(active smoker)                      |


### The stages of this problem can be decomposed as follows:
1. Data Preparation
* Ensure data is in correct format (numerical and not string)
* Normalize the data for better convergence (optional)
* Split the data into train/test subsets

2. Model Selection
* Instanciate models and fit on training data
* Evaluate model performance on testing data
* Select model with best performance

3. Submit model
* Your model will be evaluated on data that is kept separate from training/testing data
* The predictions from your model will be uploaded to the course server where it will be evaluated

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-03/
!unzip -o data.zip
!echo 'data/*' > .gitignore

Mounted at /content/gdrive
/content/gdrive/MyDrive/4c16-labs/code/lab-03
Archive:  data.zip
   creating: data/
  inflating: data/data.csv           
  inflating: data/validation.csv     


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [3]:
# Data Preparation
# The data is supplied as ".csv" format
# There are 23 columns in csv file, with 22 columns being features and 1 column being the target

# The csv file can be read into as a dataframe using the pandas library
DataFrame = pd.read_csv('data/data.csv')

# Shuffle the data
DataFrame = DataFrame.sample(frac=1)

# Visualize the first 5 rows of the imported dataframe
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
28632,Male,60,170,80,100.0,0.5,0.7,1.0,1.0,113.0,...,46.0,101.0,75.0,16.4,1.0,1.0,21.0,24.0,33.0,1.0
44369,Male,45,165,65,80.0,1.0,1.0,1.0,1.0,128.0,...,64.0,163.0,51.0,14.9,1.0,0.9,17.0,18.0,16.0,0.0
35185,Female,35,155,50,72.0,1.2,1.0,1.0,1.0,103.0,...,65.0,138.0,49.0,13.0,1.0,0.5,15.0,13.0,10.0,0.0
35136,Female,40,160,50,66.0,1.5,0.9,1.0,1.0,118.0,...,75.0,131.0,90.0,12.6,1.0,0.9,17.0,10.0,9.0,1.0
2275,Male,40,180,95,100.0,1.2,1.0,1.0,1.0,160.0,...,60.0,100.0,180.0,16.3,1.0,0.8,28.0,27.0,81.0,1.0
13975,Male,25,175,95,90.0,1.5,1.5,1.0,1.0,143.0,...,55.0,125.0,103.0,16.3,1.0,1.0,26.0,30.0,78.0,1.0


In [4]:
# The column "identified_gender" contains non-numeric data. This cannot be used as is and needs to be converted to numerical representations
# We can encode the identified gender to a numerical representation by letting "female" == 0, and "male" == 1.
# This can be done manually by using conditional statements on the dataframe, but luckily for us there is a simpler method
# We can use the built in LabelEncoder function of sklearn to do this

LabelEncoder = sklearn.preprocessing.LabelEncoder()
LabelEncoder.fit(DataFrame['identified_gender'])
DataFrame['identified_gender'] = LabelEncoder.transform(DataFrame['identified_gender'])

# Label encoder sorts the input column of data in alphabetical order, and then assigns a numerical value to each unique entry
# This results in 'female' being mapped to 0, 'male' being mapped to 1
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
28632,1,60,170,80,100.0,0.5,0.7,1.0,1.0,113.0,...,46.0,101.0,75.0,16.4,1.0,1.0,21.0,24.0,33.0,1.0
44369,1,45,165,65,80.0,1.0,1.0,1.0,1.0,128.0,...,64.0,163.0,51.0,14.9,1.0,0.9,17.0,18.0,16.0,0.0
35185,0,35,155,50,72.0,1.2,1.0,1.0,1.0,103.0,...,65.0,138.0,49.0,13.0,1.0,0.5,15.0,13.0,10.0,0.0
35136,0,40,160,50,66.0,1.5,0.9,1.0,1.0,118.0,...,75.0,131.0,90.0,12.6,1.0,0.9,17.0,10.0,9.0,1.0
2275,1,40,180,95,100.0,1.2,1.0,1.0,1.0,160.0,...,60.0,100.0,180.0,16.3,1.0,0.8,28.0,27.0,81.0,1.0
13975,1,25,175,95,90.0,1.5,1.5,1.0,1.0,143.0,...,55.0,125.0,103.0,16.3,1.0,1.0,26.0,30.0,78.0,1.0


In [5]:
# We should now separate the input features from the target feature and store them as different variables
# We do this by slicing what columns of data we want from the dataframe
# Let's first see the columns available to us by printing DataFrame.columns
print(DataFrame.columns)

Index(['identified_gender', 'age', 'height', 'weight', 'waistline',
       'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP',
       'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride',
       'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST',
       'SGOT_ALT', 'gamma_GTP', 'SMK_stat_type_cd'],
      dtype='object')


In [6]:
# The column titled "SMK_stat_type_cd" is our target, and all remaining variables are our input features.
features_columns = DataFrame.columns[:-1]
target_column = DataFrame.columns[-1:]

DataFrame_X = DataFrame[features_columns]   # Selects only Input features
DataFrame_Y = DataFrame[target_column]      # Selects only Target variable

In [7]:
# Split Data into Train/Test split
# Change the variable "test_frac" to reflect the percentage/fraction of test data in the resulting split
test_frac = 0.15

Train_X, Test_X, Train_Y, Test_Y = train_test_split(DataFrame_X, DataFrame_Y, test_size=test_frac)
print(f"Number of rows for Training:\t{Train_X.shape[0]}\nNumber of rows for Testing:\t{Test_X.shape[0]}")

Number of rows for Training:	42500
Number of rows for Testing:	7500


In [ ]:
# EDIT THIS CELL
# Construct a dictionary of prediction models to compare
# Uncomment the below dictionary and insert as many prediction models as you like.
# You may have used binary classification models in previous exercises
# You may also have to import these modules/libraries to be able to use them

# PredictionModels = {
#     'Logistic Regression': code.for.logisitic.regression.model.here(),
#     'K-NN Classificaiton': code.for.KNN.classification.model.here(),
# }

models = list(PredictionModels.keys())

print("Fitting models, this may take a while")
for _model in models:
    # This loop goes through the models in the variable "PredictionModels"
    # Complete the below code block to fit to the model to training data "Train_X, Train_Y"
    # Peform predictions on the fitted model on the Train set and Test set
    # compute f1 score


In [ ]:
# Select the model with best f1 score and write down the
# corresponding 'key' value from the 'PredictionModels' variable
# Eg if Logistic Regression is chosen then: chosen_model = 'Logistic Regression'
chosen_model = ''

In [ ]:
# DO NOT EDIT.
# Generate predictions using "chosen_model" and save to file

backend_features = pd.read_csv('data/validation.csv')
backend_features['identified_gender'] = LabelEncoder.transform(backend_features['identified_gender'])
backend_preds = PredictionModels[chosen_model].predict(backend_features)
np.savez_compressed('lab3_ex3_preds', lab3_model=backend_preds)

# Remember to push your changes to the git server for marking!